In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils import data
import torch.optim as optim
batch_size = 64

def load_data_fashion_mnist(batch_size, resize=None): 
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307, ), (0.3081, ))])
    mnist_train = datasets.FashionMNIST(
        root="../data", train=True, transform=transform, download=True)
    mnist_test = datasets.FashionMNIST(
        root="../data", train=False, transform=transform, download=True)
    return data.DataLoader(mnist_train, batch_size, shuffle=True),data.DataLoader(mnist_test, batch_size, shuffle=False)
# design model using class
 
train_loader,test_loader= load_data_fashion_mnist(batch_size, resize=28)
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv2d1=nn.Conv2d(1, 96, kernel_size=11, padding=5)
        self.conv2d2=nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv2d3=nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv2d4=nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.conv2d5=nn.Conv2d(384, 256, kernel_size=3, padding=1)
      
        self.maxpool=nn.MaxPool2d(2)
        
        self.drop=nn.Dropout(p=0.5)
        
        self.l1=nn.Linear(2304,1152)
        self.l2=nn.Linear(1152,1152)
        self.l3=nn.Linear(1152,10)
    def forward(self,x):
        in_size=x.size(0)
        x=F.relu(self.maxpool(self.conv2d1(x)))
        x=F.relu(self.maxpool(self.conv2d2(x)))
        x=F.relu(self.conv2d3(x))
        x=F.relu(self.conv2d4(x))
        x=F.relu(self.conv2d5(x))
        x=self.maxpool(x)
        x = x.view(in_size, -1)
        #print(x.shape)
        x=self.drop(F.relu(self.l1(x)))
        x=self.drop(F.relu(self.l2(x)))
        x=self.l3(x)
        return x


model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 

def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

D:\csrgzn\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


[1,   300] loss: 2.291
[1,   600] loss: 1.303
[1,   900] loss: 0.825
accuracy on test set: 71 % 
[2,   300] loss: 0.691
[2,   600] loss: 0.641
[2,   900] loss: 0.596
accuracy on test set: 77 % 
[3,   300] loss: 0.542
[3,   600] loss: 0.496
[3,   900] loss: 0.467
accuracy on test set: 82 % 
[4,   300] loss: 0.440
[4,   600] loss: 0.433
[4,   900] loss: 0.392
accuracy on test set: 84 % 
[5,   300] loss: 0.376
[5,   600] loss: 0.376
[5,   900] loss: 0.362
accuracy on test set: 85 % 
[6,   300] loss: 0.351
[6,   600] loss: 0.334
[6,   900] loss: 0.324
accuracy on test set: 87 % 
[7,   300] loss: 0.317
[7,   600] loss: 0.313
[7,   900] loss: 0.308
accuracy on test set: 87 % 
[8,   300] loss: 0.290
[8,   600] loss: 0.289
[8,   900] loss: 0.292
accuracy on test set: 86 % 
[9,   300] loss: 0.277
[9,   600] loss: 0.272
[9,   900] loss: 0.270
accuracy on test set: 88 % 
[10,   300] loss: 0.252
[10,   600] loss: 0.258
